# Grover's algorithm


In this notebook, we'll have a look at Grover's famous unstructured search algorithm. The problem is quite straightforward: we have some function $f(|x\rangle) \to \{0,1\}$ and we know that for a unique $|\omega\rangle$, $f(|\omega\rangle)=1$, in all other cases $|x\rangle \neq |\omega\rangle \iff f(|x\rangle)=0$. The goal is to find $|\omega\rangle$ given access to $f$. This is a very general and common problem: we are looking for a single item in a list, and the item is identified by the function $f$. The surprising advantage of Grover's algoritm is that it finds $|\omega\rangle$ with high probability in $O(\sqrt{N})$, (where $N=2^n$, and $n$ the number of qubits). 

Classically, you would expect that, in a list of $N$ elements, you need, in the worst case, to check the $N$ elements to find the desired item with certainty. Even if we expand this to probabilistic search, the quantum algorithm performs better: in the classical case, if we look at $N/2$ items, we have a $50\%$ chance of finding the item. In no classical scenario could we have $>50\%$ chance of finding our item by checking $\sqrt{N} \leq N/2$ items ($N\geq 4$), if you don't know anything about the properties of $f$. 

Grover's algorithm also does not hinge on the Quantum Fourier Transform, which sets it apart from Quantum Phase Estimation based algorithm, such as Shor's and the like. That's another reason why it's a very nice algorithm to study.

Habitual incantations inbound:

In [ ]:
%pip install qiskit[visualization] --quiet
%pip install matplotlib --quiet

Okay, let's see how Grover's search functions. 

We are given the _oracle_ for $f$, $U_\omega$, whereas $f$ returns $1$ for the right item, in this case we have

$$\begin{cases}U_\omega |x\rangle = -|x\rangle & \mathrm{if}\,  |x\rangle = |\omega\rangle \\ U_\omega |x\rangle = |x\rangle & \mathrm{else} \end{cases}$$

which is qualitatively similar: $U_\omega$, like $f$, can be used to check whether our current state is the solution.

You saw in last week's exercises that the operator $R$, called the _diffuser_, defined as 

$$R = 2H^{\otimes n} |0...0\rangle\langle 0...0| H^{\otimes n} - \mathbf{I}$$

performs a flip of the state's amplitudes about the mean of all amplitudes. 

Let's take a look at a two qubit case for simplicity, and assume $|01\rangle$ is the item we're looking for:

- We apply Hadamards on all qubit and get $\frac12 (|00\rangle+|01\rangle+|10\rangle+|11\rangle)$
- Applying $U_{|01\rangle}$ nets us $\frac12 (|00\rangle-|01\rangle+|10\rangle+|11\rangle)$
- The mean of all amplitudes is then $\frac14(\frac12 -\frac12 +\frac12 +\frac12) = \frac14$
- If we apply $R$, we flip amplitudes about $\frac14$: for $\frac12$ we have $\frac14 + (\frac14-\frac12)=0$, and $-\frac12$ we have $ \frac14 + (\frac14-(-\frac12))=1$ 
- In the end, the only remaining bitstring is $|01\rangle$

We see that in that case the solution $|01\rangle$ has been picked out among all items.

Let's implement Grover's algorithm.

In this case, we work with the index of the bitstring, since every bitstring is also the binary representation of some integer. For example, $|10\rangle = 0 \cdot 2^0 + 1 \cdot 2^1 = 2$ (don't forget the right-to-left ordering.) I've already written the function to get from an `int` to a bitstring, and vice-versa. 

We start with the two main blocks: $U_\omega$ and $R$. Of course, you must implement them yourself. 

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import UnitaryGate


def from_bitstring(n_qubits:int,bitstring:str)->int:
    if isinstance(bitstring,str):
        ind=np.sum([2**(n_qubits-1-int(b)) for b in bitstring])
        return ind
    else:
        return bitstring
    

def to_bitstring(n_qubits:int,ind:str)->int:
    return format(ind, "0{lp}b".format(lp=n_qubits))
    
    
# this is U_omega, the oracle
# HINT: It's really easy to flip the phase of a bitstring if you know its index
# which is why the argument is the index itself.
def oracle(n_qubits:int,target_bitstring_ind:int):
    # WRITE U_omega!
    U_om = ...
    return UnitaryGate(U_om,label="oracle")

    
# this is R, the diffuser
# this one doesn't take any index
# HINT: you can use the oracle to implement the identity matrix
# HINT 2: this is a circuit rather than a single gate because you need 
# more than a single UnitaryGate in this case
def diffuser(n_qubits:int):
    # WRITE R!
    circ = QuantumCircuit(n_qubits, name='diffuser')

    return circ
    

Now that we have the two most important tools for our algorithm, we can simply run the algorithm.

We know the optimal number of times we should run the oracle-diffuser pair; it is

$$r=\left\lfloor \frac{\pi}{4} \sqrt{N}\right\rfloor$$

(read Nielsen-Chuang 6.1.4 for a derivation of this bound.)

Let's get everything together and see if it works.

In [ ]:
from qiskit.quantum_info import Statevector

def get_most_sampled_bitstring(circ):
    statevector = Statevector.from_instruction(circ)
    counts=statevector.sample_counts(1000)
    bitstring =counts.most_frequent()
    return bitstring


def grovers_algorithm(n_qubits, target_bitstring:int|str):
    
    circ = QuantumCircuit(n_qubits, n_qubits)
    bitstring_ind = from_bitstring(n_qubits,target_bitstring)

    r = int(np.floor(np.pi/4*np.sqrt(2**n_qubits)))
    circ.h(range(n_qubits))
    
    for _ in range(r):
        circ.append(oracle(n_qubits, bitstring_ind), range(n_qubits)) #U
        circ.append(diffuser(n_qubits), range(n_qubits)) # R
  
    return circ

n_qubits = 6

for trial in range(10):
    random_bitstring_ind=np.random.randint(0,2**n_qubits)
    solution=to_bitstring(n_qubits,random_bitstring_ind)
    circ = grovers_algorithm(n_qubits, random_bitstring_ind)
    guess = get_most_sampled_bitstring(circ)
    print(f"trial {trial}: solution: {solution}, guess: {guess} match: {guess==solution}")

Note that you can also expand Grover's algorithm to the case where you look for multiple items. You only need minor changes: the oracle flips all the solutions, and the number of repeats is reduced by a factor $\frac{1}{\sqrt{k}}$ where $k$ is the number of items you're looking for.